## Conventions
* スペルミス，大文字，小文字，句読点の間違い

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### pyspellchecker
* 句読点を全て削除してからが良さそう

In [4]:
%%capture
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00a 0:00:01


In [5]:
from spellchecker import SpellChecker

spell = SpellChecker()

In [6]:
misspelled = spell.unknown(['something', 'is', 'hapenning', 'here'])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

happening
{'happening', 'henning', 'penning'}


In [9]:
df = pd.read_csv('/workspace/kaggle_data/train.csv')
text = df['full_text']

In [13]:
text[0]

"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear.\n\nmost students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go t

In [72]:
# 改行の削除
new_text = text[0].replace('\n', '')
# 句読点を抜く
new_text = new_text.replace('.', ' ')
new_text = new_text.replace(',', ' ')

new_text = new_text.replace('!', '')
new_text = new_text.replace('?', '')
new_text = new_text.replace('"', '')

new_text = new_text.replace(':', '')
new_text = new_text.replace(';', '')

# 文字列に数字が入るもの

In [73]:
new_text

"I think that students would benefit from learning at home because they wont have to change and get up early in the morning to shower and do there hair  taking only classes helps them because at there house they'll be pay more attention  they will be comfortable at home The hardest part of school is getting ready  you wake up go brush your teeth and go to your closet and look at your cloths  after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain  Then you'll have to change  with the online classes you can wear anything and stay home and you wont need to stress about what to wear most students usually take showers before school  they either take it before they sleep or when they wake up  some students do both to smell good  that causes them do miss the bus and effects on there lesson time cause they come late to school  when u have online classes u wont need to miss lessons cause you can get everything set up and go take a sh

In [74]:
text_words = new_text.split()
print(text_words)

['I', 'think', 'that', 'students', 'would', 'benefit', 'from', 'learning', 'at', 'home', 'because', 'they', 'wont', 'have', 'to', 'change', 'and', 'get', 'up', 'early', 'in', 'the', 'morning', 'to', 'shower', 'and', 'do', 'there', 'hair', 'taking', 'only', 'classes', 'helps', 'them', 'because', 'at', 'there', 'house', "they'll", 'be', 'pay', 'more', 'attention', 'they', 'will', 'be', 'comfortable', 'at', 'home', 'The', 'hardest', 'part', 'of', 'school', 'is', 'getting', 'ready', 'you', 'wake', 'up', 'go', 'brush', 'your', 'teeth', 'and', 'go', 'to', 'your', 'closet', 'and', 'look', 'at', 'your', 'cloths', 'after', 'you', 'think', 'you', 'picked', 'a', 'outfit', 'u', 'go', 'look', 'in', 'the', 'mirror', 'and', 'youll', 'either', 'not', 'like', 'it', 'or', 'you', 'look', 'and', 'see', 'a', 'stain', 'Then', "you'll", 'have', 'to', 'change', 'with', 'the', 'online', 'classes', 'you', 'can', 'wear', 'anything', 'and', 'stay', 'home', 'and', 'you', 'wont', 'need', 'to', 'stress', 'about', 'w

In [ ]:
# 正しいのにカウントされているもの
# you're
# tv
# 'がついているもの
# `がついている
# 1文字のもの
# シングルクオートがついている省略表現を列挙

In [125]:
str1 = "12345"

any(chr.isdigit() for chr in str1)

True

#### ミススペルをカウントする関数

In [220]:
def misspell_count(text):
    # 改行の削除
    text = text.replace('\n', '')
    # 句読点を抜く
    text = text.replace('.', ' ')
    text = text.replace(',', ' ')
    text = text.replace('!', ' ')
    text = text.replace('?', ' ')
    # ダブルクォーテーションで囲まれているものの中身だけ取り出す
    #text = text.strip('"')
    text = text.replace('"', ' ')
    # シングルクオートをそのまま削除するのは上手くいかない
    #text = text.replace("''", ' ')   
    text = text.replace(':', ' ')
    text = text.replace(';', ' ')
    text = text.replace('/', ' ')
    text = text.replace('-', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('_', ' ')
    #text = text.replace('\', ' ')
    
    # 固有名詞も除きたいかも
    
    text_words = text.split()
    misspelled = spell.unknown(text_words)
    count = 0
    for word in misspelled:
        # 一文字をミススペルに入れた方が相関係数が高い
        #if len(word) == 1:
         #   count = count
        # 数字が含まれる時はカウントしない
        # 数字が含まれているものも未知語としてカウントされるため
        if any(chr.isdigit() for chr in word):
            count = count
        else:
            count += 1
            # スペルミス
            #print(word)
        # 最も可能性の高い単語
        #print(spell.correction(word))
    #print(count)
    return count 

In [221]:
miss_count = []

for i in range(3911):
    check_text = text[i]
    m_count = misspell_count(check_text)
    miss_count.append(m_count)

#miss_count

#### 各目的変数ごとの平均

In [222]:
df['miss_count'] = miss_count

In [223]:
df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,miss_count
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,1
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,4
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,1
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,7


In [224]:
df.groupby(['conventions'])['miss_count'].mean()

conventions
1.0    21.666667
1.5    11.900000
2.0     9.965174
2.5     5.971939
3.0     3.847958
3.5     2.648678
4.0     1.712810
4.5     1.877049
5.0     2.240000
Name: miss_count, dtype: float64

In [225]:
df.groupby(['grammar'])['miss_count'].mean()

grammar
1.0    16.500000
1.5    14.000000
2.0     6.613971
2.5     5.286550
3.0     4.191147
3.5     3.323864
4.0     2.684564
4.5     2.298507
5.0     2.413793
Name: miss_count, dtype: float64

In [226]:
df.groupby(['phraseology'])['miss_count'].mean()

phraseology
1.0    11.300000
1.5     9.636364
2.0     7.400000
2.5     5.480570
3.0     4.301821
3.5     3.400431
4.0     3.016275
4.5     2.648148
5.0     3.440000
Name: miss_count, dtype: float64

In [227]:
df.groupby(['vocabulary'])['miss_count'].mean()

vocabulary
1.0     4.000000
1.5    15.285714
2.0     8.395161
2.5     6.357955
3.0     4.607452
3.5     3.459782
4.0     3.102253
4.5     2.104348
5.0     3.365854
Name: miss_count, dtype: float64

In [228]:
df.groupby(['syntax'])['miss_count'].mean()

syntax
1.0    14.727273
1.5    12.103448
2.0     6.439024
2.5     5.640048
3.0     4.096800
3.5     3.230681
4.0     2.948454
4.5     2.060000
5.0     2.470588
Name: miss_count, dtype: float64

In [229]:
df.groupby(['cohesion'])['miss_count'].mean()

cohesion
1.0    22.400000
1.5    11.259259
2.0     5.869841
2.5     5.603797
3.0     4.344891
3.5     3.733806
4.0     3.011236
4.5     2.088000
5.0     2.884615
Name: miss_count, dtype: float64

#### 相関係数

In [230]:
miss_count = np.array(miss_count)
col_list = ['cohesion', 'syntax', 'vocabulary','phraseology', 'grammar', 'conventions']
for i in col_list:
    conv = df[i].to_numpy()
    print(np.corrcoef(miss_count, conv))

[[ 1.         -0.20602587]
 [-0.20602587  1.        ]]
[[ 1.         -0.22838238]
 [-0.22838238  1.        ]]
[[ 1.         -0.21791612]
 [-0.21791612  1.        ]]
[[ 1.         -0.21424831]
 [-0.21424831  1.        ]]
[[ 1.         -0.23731966]
 [-0.23731966  1.        ]]
[[ 1.         -0.40303354]
 [-0.40303354  1.        ]]


* 最もたかい相関係数  0.40303354

In [ ]:
fig, ax = plt.subplots(figsize=(8.0, 10.0))
#fig = plt.figure(figsize=(14,8))
#ax = sns.jointplot(x = conv, y = miss_count, color = "#6495ED")
ax.scatter(conv, miss_count)
ax.invert_xaxis()

In [86]:
misspelled = spell.unknown(text_words)

for word in misspelled:
    # スペルミス
    print(word)
    # 最も可能性の高い単語
    print(spell.correction(word))

In [24]:
misspelled

set()

### PyEnchant(使えていない)
* pythonのスペルチェッカー

In [2]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main arm64 Packages [375 kB]
Get:5 http://deb.debian.org/debian buster/main arm64 Packages [7737 kB]
Get:6 http://deb.debian.org/debian buster-updates/main arm64 Packages [8780 B]
Fetched 8334 kB in 1s (7703 kB/s)3m                       
Reading package lists... Done
Building dependency tree       
Reading state information... Done
109 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.7-0 libtext-iconv-perl
Suggested packages:
  aspell-do

In [8]:
#%%capture
!pip install pyenchant

In [3]:
import enchant
d = enchant.Dict("en_US")
d.check("Hello")

ImportError: The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details


In [ ]:
d.check("Helo")